In [1]:
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
import gc, os
import time
from datetime import datetime
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

generate='./data/generate/'
path='./data/rec_data/'

In [2]:
data=pd.read_feather(generate+'DIN_data.feather')

In [3]:
del data['hist_itemId']
del data['hist_category']

In [4]:
sparse_feats = ['pvId','suv','operator', 'browserType', 'deviceType', 'osType', 'province','city'] + ['itemId','category']

In [5]:
for f in sparse_feats:
        data[f+'_count'] = data[f].map(data[f].value_counts())

In [6]:
nunique_group = []
key_feature = ['pvId','suv']
target_feature = ['itemId','category']

In [7]:
def group_fea(df, key, target):
    # 计算每一个key和多少个不同的target交互
    tmp = df.groupby(key, as_index=False)[target].agg({
        key + '_' + target + '_nunique': 'nunique',
    }).reset_index().drop('index', axis=1)

    return tmp

In [8]:
for feat_1 in key_feature:
    for feat_2 in target_feature:
        if feat_1 + '_' + feat_2 + '_nunique' not in nunique_group:
            nunique_group.append(feat_1 + '_' + feat_2 + '_nunique')
            tmp = group_fea(data, feat_1, feat_2)
            data = data.merge(tmp, on=feat_1, how='left')

        if feat_2 + '_' + feat_1 + '_nunique' not in nunique_group:
            nunique_group.append(feat_2 + '_' + feat_1 + '_nunique')
            tmp = group_fea(data, feat_2, feat_1)
            data = data.merge(tmp, on=feat_2, how='left')

In [9]:
key_feature = ['itemId']
target_feature = ['pvId','suv','operator', 'browserType', 'deviceType', 'osType', 'province','city']

for feat_1 in key_feature:
    for feat_2 in target_feature:
        if feat_1 + '_' + feat_2 + '_nunique' not in nunique_group:
            nunique_group.append(feat_1 + '_' + feat_2 + '_nunique')
            tmp = group_fea(data, feat_1, feat_2)
            data = data.merge(tmp, on=feat_1, how='left')

        if feat_2 + '_' + feat_1 + '_nunique' not in nunique_group:
            nunique_group.append(feat_2 + '_' + feat_1 + '_nunique')
            tmp = group_fea(data, feat_2, feat_1)
            data = data.merge(tmp, on=feat_2, how='left')

In [10]:
data.to_feather(generate+'new_data.feather')

In [11]:
data.columns

Index(['label', 'pvId', 'suv', 'itemId', 'operator', 'browserType',
       'deviceType', 'osType', 'province', 'city', 'category', 'pvid_count',
       'pvid_item_nunique', 'pvid_suv_nunique', 'item_count',
       'item_suv_nunique', 'pvId_count', 'suv_count', 'operator_count',
       'browserType_count', 'deviceType_count', 'osType_count',
       'province_count', 'city_count', 'itemId_count', 'category_count',
       'pvId_itemId_nunique', 'itemId_pvId_nunique', 'pvId_category_nunique',
       'category_pvId_nunique', 'suv_itemId_nunique', 'itemId_suv_nunique',
       'suv_category_nunique', 'category_suv_nunique',
       'itemId_operator_nunique', 'operator_itemId_nunique',
       'itemId_browserType_nunique', 'browserType_itemId_nunique',
       'itemId_deviceType_nunique', 'deviceType_itemId_nunique',
       'itemId_osType_nunique', 'osType_itemId_nunique',
       'itemId_province_nunique', 'province_itemId_nunique',
       'itemId_city_nunique', 'city_itemId_nunique'],
      dtyp

In [12]:
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
import gc, os
import time
from datetime import datetime
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

generate='./data/generate/'
path='./data/rec_data/'

In [13]:
lgb_cols=['pvId', 'suv', 'itemId', 'operator', 'browserType',
       'deviceType', 'osType', 'province', 'city', 'category', 'pvid_count',
       'pvid_item_nunique', 'item_count', 'pvId_count', 'suv_count', 'operator_count',
       'browserType_count', 'deviceType_count', 'osType_count',
       'province_count', 'city_count', 'itemId_count', 'category_count',
       'pvId_itemId_nunique', 'itemId_pvId_nunique', 'pvId_category_nunique',
       'category_pvId_nunique', 'suv_itemId_nunique', 'itemId_suv_nunique',
       'suv_category_nunique', 'category_suv_nunique',
       'itemId_operator_nunique', 'operator_itemId_nunique',
       'itemId_browserType_nunique', 'browserType_itemId_nunique',
       'itemId_deviceType_nunique', 'deviceType_itemId_nunique',
       'itemId_osType_nunique', 'osType_itemId_nunique',
       'itemId_province_nunique', 'province_itemId_nunique',
       'itemId_city_nunique', 'city_itemId_nunique']

In [14]:
# 数据处理sparse、dense
for feat in ['pvId','suv']:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
    
train=data[data['label']!=9]
val=data[data['label']==9]
del data

In [15]:
train['pvid_count_new'] = train.groupby('pvId')['itemId'].transform('count')

In [16]:
tmp=train[['pvId','pvid_count_new']]
tmp=tmp.drop_duplicates(keep='first')

In [17]:
g_train = tmp['pvid_count_new'].values.tolist()

In [18]:
len(g_train)

299919

In [19]:
del train['pvid_count_new']

In [20]:
# 模型及参数的定义
lgb_Classfication = lgb.LGBMRanker(boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,objective='lambdarank',
                            max_depth=-1, n_estimators=1500, subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
                            learning_rate=0.01, min_child_weight=50, random_state=2022, n_jobs= 16)

In [ ]:
lgb_Classfication.fit(train[lgb_cols], train['label'],group=g_train)

In [ ]:
# 模型预测
val['rate'] = lgb_Classfication.predict(val[lgb_cols])

In [ ]:
label_raw=pd.read_csv(path+'rec-train-dataset.csv',usecols=['label'])
label_raw=label_raw['label'].tolist()
val['label']=label_raw

In [ ]:
ygq=val[['label','pvId','rate']]

In [ ]:
from sklearn.metrics import roc_auc_score
from collections import defaultdict
from tqdm import tqdm
def uAUC(labels, preds, pvId_list):
    user_pred = defaultdict(lambda: [])
    user_truth = defaultdict(lambda: [])
    for idx, truth in enumerate(labels):
        pvId = pvId_list[idx]
        pred = preds[idx]
        truth = labels[idx]
        user_pred[pvId].append(pred)
        user_truth[pvId].append(truth)
    total_auc = 0.0
    size = 0.0
    
    pvId_flag = defaultdict(lambda: False)
    for pvId in set(pvId_list):
        truths = user_truth[pvId]
        flag = False
        for i in range(len(truths) - 1):
            if truths[i] != truths[i + 1]:
                flag = True
                break
        pvId_flag[pvId] = flag

    for pvId in tqdm(set(pvId_list)):
        if pvId_flag[pvId]:
            auc = roc_auc_score(np.asarray(user_truth[pvId]), np.asarray(user_pred[pvId]))
            total_auc += auc 
            size += 1.0
    # size = max(size, 1)
    user_auc = float(total_auc)/size
    return user_auc

In [ ]:
print(uAUC(ygq['label'].values.tolist(), ygq['rate'].values.tolist(), ygq['pvId'].values.tolist()))

In [ ]:
ygq

In [ ]:
df['aa']=lgb_Classfication.feature_importances_()

In [ ]:
imp_df = pd.DataFrame()
imp_df['feats'] = lgb_Classfication.feature_name_
imp_df['imp'] = lgb_Classfication.feature_importances_
imp_df.sort_values('imp', ascending=False, inplace=True)
imp_df.to_csv('./imp_df1.csv')